In [1]:
import lancedb
from lancedb.pydantic import LanceModel, Vector

/Users/raghavdixit/Desktop/open_source/lancedb/python/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
! rm -rf /tmp/lancedb

In [6]:
tmp_path = "/tmp/lancedb"

In [7]:
db = lancedb.connect(tmp_path)
assert db.uri == str(tmp_path)
assert db.table_names() == []

class SimpleModel(LanceModel):
    item: str
    price: float
    vector: Vector(2)

table = db.create_table(
    "test",
    data=[
        {"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
        {"vector": [5.9, 26.5], "item": "bar", "price": 20.0},
    ],
    schema=SimpleModel,
)
try:
    table.add(SimpleModel(item="baz", price=30.0, vector=[1.0, 2.0]))
except Exception as e:
    assert type(e) == ValueError
    assert str(e) == "Cannot add a single LanceModel to a table. Use a list."

rs = table.search([100, 100]).limit(1).to_pandas()
assert len(rs) == 1
assert rs["item"].iloc[0] == "bar"

assert db.table_names() == ["test"]
assert "test" in db
assert len(db) == 1

assert db.open_table("test").name == db["test"].name

NameError: name 'LanceModel' is not defined